In [1]:
import pycaret
import pandas as pd

ModuleNotFoundError: No module named 'pycaret'

In [101]:
# อ่านไฟล์ CSV
data = pd.read_csv('export_data/filtered_data_3_best.csv')

In [102]:
# ตั้ง 'timestamp' เป็น index หากยังไม่ได้ทำ
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# ตั้งค่า freq ให้กับ index
data = data.asfreq('D')  # 'D' สำหรับรายวัน

In [103]:
data.tail()

,humidity,pm_10,pm_2_5,pm_2_5_sp,temperature
timestamp,,,,,
2025-02-07 07:00:00,70.933952,34.743259,29.321219,26.166471,28.271927
2025-02-08 07:00:00,69.348548,30.654249,26.000000,22.380675,28.385596
2025-02-09 07:00:00,69.916384,28.971065,24.616898,20.976852,28.373742
2025-02-10 07:00:00,71.226336,30.652667,25.856981,22.374574,28.251201
2025-02-11 07:00:00,68.159624,32.532955,27.522727,24.120455,28.693799


In [104]:
data.head()

,humidity,pm_10,pm_2_5,pm_2_5_sp,temperature
timestamp,,,,,
2023-07-01 07:00:00,67.844074,12.634307,11.081056,8.796418,31.131941
2023-07-02 07:00:00,68.552865,11.350789,9.705165,7.547346,32.240924
2023-07-03 07:00:00,63.142228,13.457203,11.469033,8.783577,35.841610
2023-07-04 07:00:00,61.741030,29.167827,26.188719,25.657382,36.269393
2023-07-05 07:00:00,62.085527,28.747735,25.508711,23.605575,36.176134


In [105]:
# แสดงแถวที่มีค่าหายไปก่อนเติม
print("แถวที่มีค่าหายไปก่อนเติม:")
print(data[data.isnull().any(axis=1)])

# เติมค่าที่หายไปด้วยการ interpolate แบบเส้นตรง
data.interpolate(method='linear', inplace=True)

# แสดงแถวที่ยังมีค่า NaN หลังจากเติม (ถ้ามี)
missing_rows_after = data[data.isnull().any(axis=1)]
if not missing_rows_after.empty:
    print("แถวที่ยังมีค่าหายไปหลังเติม:")
    print(missing_rows_after)
else:
    print("ไม่มีค่าหายไปแล้ว")

row = data.loc["2023-09-30 07:00:00"]
row

แถวที่มีค่าหายไปก่อนเติม:
                     humidity  pm_10  pm_2_5  pm_2_5_sp  temperature
timestamp                                                           
2023-09-30 07:00:00       NaN    NaN     NaN        NaN          NaN
ไม่มีค่าหายไปแล้ว


humidity       82.889367
pm_10          60.525519
pm_2_5         53.415241
pm_2_5_sp      69.527667
temperature    29.219760
Name: 2023-09-30 07:00:00, dtype: float64

In [106]:
# # สร้างช่วงเวลาใหม่ 30 วันจากวันสุดท้ายของข้อมูลเดิม
# last_date = data.index[-1]  # หาวันสุดท้ายของข้อมูลเดิม
# new_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=30, freq='D')

# # สร้าง DataFrame ใหม่สำหรับวัน 30 วันที่เพิ่มเข้ามา
# new_data = pd.DataFrame({
#     'pm_2_5': [None] * 30  # สมมติว่าใช้ None หรือ NaN สำหรับค่าในอนาคต
# }, index=new_dates)

# # รวมข้อมูลเดิมกับข้อมูลใหม่
# data_with_future = pd.concat([data, new_data])

# # แสดงข้อมูลที่เพิ่มวัน 30 วัน
# print("\nข้อมูลหลังเพิ่ม 30 วัน:")
# print(data_with_future)

In [ ]:
from pycaret.time_series import *
# ตั้งค่าของ PyCaret
s = setup(data=data, target='pm_2_5', fh=30, session_id=123)

,Description,Value
0,session_id,123
1,Target,pm_2_5
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(592, 5)"
5,Transformed data shape,"(592, 5)"
6,Transformed train set shape,"(562, 5)"
7,Transformed test set shape,"(30, 5)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [109]:
pycaret.__version__

'3.3.2'

In [110]:
# compare baseline models
best = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,0.0243,0.0248,0.1651,0.2256,0.0130,0.0129,0.9991,3.8633
arima,ARIMA,0.0432,0.0423,0.2948,0.3867,0.0290,0.0298,0.9948,0.0567
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.0762,0.0651,0.5197,0.5937,0.0456,0.0449,0.9909,0.1567
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.0898,0.0738,0.6124,0.6728,0.0550,0.0545,0.9885,0.1600
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.0901,0.0739,0.6140,0.6742,0.0551,0.0546,0.9885,0.1433
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.0910,0.0745,0.6203,0.6796,0.0555,0.0550,0.9883,0.1600
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.1480,0.1194,1.0085,1.0880,0.0857,0.0822,0.9714,0.2100
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.1625,0.1308,1.1074,1.1915,0.0937,0.0913,0.9660,0.3467
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.1689,0.1365,1.1507,1.2446,0.1006,0.0955,0.9611,0.3600
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.1702,0.1517,1.1594,1.3831,0.1086,0.1035,0.9525,0.1633


In [ ]:
# ตรวจสอบข้อมูลที่ใช้ในการฝึก (train) และทดสอบ (test)
train_data = get_config('X_train')  # ข้อมูลฝึก
test_data = get_config('X_test')   # ข้อมูลทดสอบ

In [ ]:
train_data

,humidity,pm_10,pm_2_5_sp,temperature
2023-07-01,67.844074,12.634307,8.796418,31.131941
2023-07-02,68.552865,11.350789,7.547346,32.240924
2023-07-03,63.142228,13.457203,8.783577,35.841610
2023-07-04,61.741030,29.167827,25.657382,36.269393
2023-07-05,62.085527,28.747735,23.605575,36.176134
...,...,...,...,...
2025-01-08,71.168637,39.411902,36.767795,28.921543
2025-01-09,70.983573,38.691514,31.946101,28.383165
2025-01-10,70.491487,42.104094,36.856140,28.945704
2025-01-11,73.568381,45.083333,42.007042,28.357459


In [ ]:
test_data

,humidity,pm_10,pm_2_5_sp,temperature
2025-01-13,78.292202,48.632215,47.726174,25.997341
2025-01-14,74.887204,58.218182,67.738182,26.780883
2025-01-15,74.201402,46.750000,45.204651,27.146722
2025-01-16,89.873425,51.154312,60.757943,24.990604
2025-01-17,80.848939,17.509220,10.900709,27.221665
2025-01-18,83.512677,20.454545,12.652893,26.832436
2025-01-19,85.338095,35.741685,30.738359,25.598781
2025-01-20,84.719634,44.953250,41.101482,26.357235
2025-01-21,72.150981,25.172840,17.993266,28.367937
2025-01-22,75.599793,35.329412,27.448235,27.671564


In [ ]:
plot_model(best, plot = 'forecast')

In [ ]:
# predict on test set
holdout_pred = predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Auto ARIMA,0.0033,0.0052,0.0229,0.0484,0.0011,0.0011,0.0000


In [ ]:
predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Auto ARIMA,0.0674,0.0752,0.3539,0.5274,132128107067671.4219,1.0128,0.9989


,y_pred
2025-01-13,43.3116
2025-01-14,52.9304
2025-01-15,41.5750
2025-01-16,46.6795
2025-01-17,15.1398
2025-01-18,17.6447
2025-01-19,31.4703
2025-01-20,39.7597
2025-01-21,21.8504
2025-01-22,30.7883


In [ ]:
# คำนวณ Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(future_forecast['pm_2_5'], future_forecast['Label'])

# คำนวณ R-Squared (R²)
r_squared = r2_score(future_forecast['pm_2_5'], future_forecast['Label'])

# แสดงผลลัพธ์
print(f"MAPE: {mape*100:.2f}%")
print(f"R-Squared: {r_squared:.2f}")

In [ ]:
# save pipeline
save_model(best, 'my_first_pipeline')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 AutoARIMA(random_state=123,
                                                                           sp=2,
                                                                           suppress_warnings=True))]))]),
 'my_first_pipeline.pkl')

In [ ]:
# show predictions df
holdout_pred.head()

,y_pred
2025-01-13,43.2594
2025-01-14,52.9259
2025-01-15,41.4550
2025-01-16,46.5637
2025-01-17,14.9130


In [ ]:
# สร้างช่วงเวลาอนาคต (90 วันข้างหน้า)
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
future_dates

DatetimeIndex(['2025-02-12 07:00:00', '2025-02-13 07:00:00',
               '2025-02-14 07:00:00', '2025-02-15 07:00:00',
               '2025-02-16 07:00:00', '2025-02-17 07:00:00',
               '2025-02-18 07:00:00', '2025-02-19 07:00:00',
               '2025-02-20 07:00:00', '2025-02-21 07:00:00',
               '2025-02-22 07:00:00', '2025-02-23 07:00:00',
               '2025-02-24 07:00:00', '2025-02-25 07:00:00',
               '2025-02-26 07:00:00', '2025-02-27 07:00:00',
               '2025-02-28 07:00:00', '2025-03-01 07:00:00',
               '2025-03-02 07:00:00', '2025-03-03 07:00:00',
               '2025-03-04 07:00:00', '2025-03-05 07:00:00',
               '2025-03-06 07:00:00', '2025-03-07 07:00:00',
               '2025-03-08 07:00:00', '2025-03-09 07:00:00',
               '2025-03-10 07:00:00', '2025-03-11 07:00:00',
               '2025-03-12 07:00:00', '2025-03-13 07:00:00'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
# สร้าง DataFrame เปล่าสำหรับข้อมูลอนาคต
future_data = pd.DataFrame(index=future_dates)
future_data

""
2025-02-12 07:00:00
2025-02-13 07:00:00
2025-02-14 07:00:00
2025-02-15 07:00:00
2025-02-16 07:00:00
2025-02-17 07:00:00
2025-02-18 07:00:00
2025-02-19 07:00:00
2025-02-20 07:00:00
2025-02-21 07:00:00


In [ ]:
predict_model(best, data=future_data)